In [1]:
from fastai import *
from fastai.vision import *
from fastai.vision.models.unet import DynamicUnet
from fastai.vision.learner import cnn_config
from fastai.callbacks import *
import sys
sys.path.append('../../')
sys.path.append('../../model')
from model.losses import *
from model.bpho.resnet import *
from model.bpho.unet import *
from model.metrics import psnr, ssim
from data.load_fluo import *

In [2]:
import matplotlib.pyplot as plt

In [3]:
gpu_id = 2
num_cores = 4
torch.cuda.set_device(gpu_id)

### data load

In [4]:
data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF/Mitotracker/training/trainsets/')

In [5]:
bs=8
lr = slice(None, 4e-4, None)
size=512 
tile_sz=512 
# datasetname=multi_100mitotracker 
cycles=100
# save_name=multit_5_100mito_opt_001 
n_frames=5

In [6]:
lr_type = 't'
if lr_type == 's':
    z_frames, t_frames = 1, 1
    n_frames = 1
    dataset = data_pth/'mitotracker_PSSR-SF'
    hr_tifs = dataset/f'hr'
    lr_tifs = dataset/f'lr'
elif lr_type == 't':
    z_frames, t_frames = 1, 5
    n_frames = 5
    multi_str = f'_{lr_type}_{n_frames}'
    tile_sz = 512
    dataset = data_pth/'mitotracker_PSSR-MF'
    hr_tifs = dataset/f'hr_t_{tile_sz:d}{multi_str}'
    lr_tifs = dataset/f'lr_t_{tile_sz:d}{multi_str}'

In [7]:
mode = 'L'
norm = True
noise = False
cutout = False # Maybe?

In [8]:
data = get_data(bs, size, lr_tifs, hr_tifs, n_frames=n_frames,  max_zoom=4., 
                    use_cutout=cutout, use_noise=noise, mode=mode, norm=norm)

normalizing x and y data


### initiate learner

In [9]:
wnres_args = {
    'blur': True,
    'blur_final': True,
    'bottle': True,
    'self_attention': True,
    'last_cross': True
    
}
arch = 'wnresnet34'
wd = 1e-3
arch = eval(arch)
metrics = [psnr, ssim]

# Lj & ours

In [10]:
model_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF/Mitotracker/models')
state_dict = torch.load(model_pth/'feat_loss'/'sf_state_dict.pkl')

In [12]:
model_name = 'mitotracker_PSSR-MF_512'
learn_lj = load_learner(path=model_pth/'baselines', file=f'{model_name}.pkl')
# learn_lj.model.load_state_dict(state_dict)

In [13]:
learn = wnres_unet_learner(data, arch, in_c=n_frames, wnres_args=wnres_args,
                         metrics=metrics,
                           model_dir=model_pth, callback_fns=[LossMetrics], wd=wd)
# learn.model.load_state_dict(state_dict)

In [ ]:
xb, yb = learn.data.one_batch()

In [ ]:
yb.shape

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(learn.data.single_ds.y.reconstruct(t=yb[0]).data[0])
plt.subplot(1, 2, 2)
plt.imshow(learn_lj.data.single_ds.y.reconstruct(t=yb[0]).data[0])
plt.show()

In [ ]:
set(lj_dict.keys()).difference(set(our_dict.keys()))

In [ ]:
learn_lj.data.single_ds??